----
<div style="display: flex; align-items: center;">
    <img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="20%" style="vertical-align: top;">
    <h1 style="margin-left: 20px;">Data Library for Python</h1>
</div>

----

## Content - Pricing - Synchronous calls

This notebook demonstrates how to use the Pricing interface to retrieve snapshot data from the LSEG Data Platform.

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the LD Library in the **_lseg-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _LD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [2]:
import os
os.environ["LD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [2]:
import lseg.data as ld
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [13]:
# Open the Default Session
ld.open_session()

<lseg.data.session.Definition object at 0x1cf1ea8b5d0 {name='workspace'}>

## Get Price Snapshot
The following gets a snapshot from the non-streaming RDP Pricing API - note the **get_data()** call

In [6]:
# Define our RDP Snapshot Price object
response = ld.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_data()

display(response.data.df)

Instrument     BID     ASK
0       EUR=  1.0722  1.0723
1       GBP=    1.27  1.2704
2       JPY=  158.41  158.44
3       CAD=  1.3717  1.3718

## Alternative method for Snapshot data
If the above code generates an error, this could be because you are not licensed for the RDP Snapshot API OR you are using a **Deployed** session.   
If this is the case, then the following can be used to request a Snapshot from the Streaming data feed (assuming you have a Streaming Data licence) - note the **get_stream()** call

In [7]:
# Define our Streaming Price object
non_streaming = ld.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()

# We want to just snap the current prices, don't need updates
# Open the instrument in non-streaming mode
non_streaming.open(with_updates=False)
# Snapshot the prices at the time of the open() call
non_streaming.get_snapshot()

Instrument
0       EUR=
1       GBP=
2       JPY=
3       CAD=

The non-streaming requests are closed by the server - **no need to close them**.   

### Alternative ways of accessing instruments + values

#### Direct Access to fields

In [8]:
non_streaming['EUR=']['BID']

In [9]:
eur = non_streaming['EUR=']
eur['ASK']

#### Iterate on fields

In [10]:
print('JPY=')
for field_name, field_value in non_streaming['JPY=']:
    print(f"\t{field_name} : {field_value}")

JPY=


#### Iterate on instruments and fields

In [11]:
for instrument in non_streaming:
    print(instrument.name)
    for field_name, field_value in instrument:
        print(f"\t{field_name} : {field_value}")

EUR=
GBP=
JPY=
CAD=


## Close the session

In [12]:
ld.close_session()